## Library

In [ ]:
import os
import requests as req
import concurrent.futures
from tqdm import tqdm 

# Authorizations Earthdata NASA

## Change Here

Earthdata Username and Password

In [ ]:
# Your credentials (replace with your username and password)
username = "your username"
password = "your password"

Earthdata NASA User Token for Authorizing Download

In [ ]:
# Replace token from Earthdata User Token
# Make sure the Token is still valid or not expired

token = 'your token'

headers = {
    'Authorization': f'Bearer {token}'
}

## Auth Init

In [ ]:
# URL of the Earthdata Login endpoint for authentication
login_url = "https://urs.earthdata.nasa.gov/login"

# Create a session to maintain cookies and headers
session = req.Session()

# Send a GET request to the login page to obtain authentication cookies
response = session.get(login_url)
        
# Prepare the POST data with your credentials
data = {
"username": username,
"password": password,
"_eventId": "submit",
"action": "login"
}

# Send a POST request to the login page with your credentials
login_response = session.post(login_url, data=data)

# POST request login check
if login_response.status_code == 200:
    print("Login successful.")
else:
    print(f"Login failed with status code: {login_response.status_code}")

# Download Functions

## Directory Change Here

URLs txt File Directory

In [ ]:
# Specify the full path to the 'url.txt' file
# Change path in the ('....')
# e.g. E:\#Project\GPM\raw_data\url.txt

url_file_path = r"directory to .txt files"

Output Save File

In [ ]:
# Specify the output folder where downloaded files will be saved. 
# Change path in the ('....')
# e.g. E:\#Project\GPM\downloaded_files

output_folder = r"directory to output folder"

# Create the output folder if it doesn't exist
# Don't Change
os.makedirs(output_folder, exist_ok=True)

## Read '.txt' URLs file and Download Functions

In [ ]:
# Download function
def download_url(url, output_folder):
    print('Downloading...')
    
    try:
        r = req.get(url, headers=headers, stream=True)
        if r.status_code == req.codes.ok:
            file_name = os.path.join(output_folder, url.split('/')[-1])               
            total_size = int(r.headers.get('content-length', 0))
            progress_bar = tqdm(total=total_size, unit='B', unit_scale=True)
                        
            with open(file_name, 'wb') as f:
                for data in r.iter_content(chunk_size=8192):
                    f.write(data)
                    progress_bar.update(len(data))
            progress_bar.close()
            
            print("Download completed")
        else:
            print("Failed to download:", url, "Status Code:", r.status_code)
    except Exception as e:
        print("An error occurred while downloading", url, "Error:", str(e))

# Read the list of URLs from a text file
with open(url_file_path, 'r') as f:
    urls = f.read().splitlines()
    
# Number of threads
max_workers = 6

# Download Proccess

In [ ]:
# Download files concurrently using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    for url in urls:
        executor.submit(download_url, url, output_folder)